In [ ]:
#import necessary libraries
import os
import tensorflow.keras as tk 
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.layers import Input 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/facial_expression/fer2013/fer2013.csv')
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
df.emotion.unique()

array([0, 2, 4, 6, 3, 5, 1])

In [ ]:
label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'} 

In [ ]:
img_array = df.pixels.apply(lambda x : np.array(x.split(' ')).reshape(48, 48,1).astype('float32'))

In [ ]:
img_array = np.stack(img_array,axis=0)

In [ ]:
lables = df.emotion.values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(img_array,lables,test_size = .2)

In [ ]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((28709, 48, 48, 1), (28709,), (7178, 48, 48, 1), (7178,))

In [ ]:
x_train = x_train/255
x_test = x_test/255

In [ ]:
model = tk.Sequential()
model.add(tk.layers.Conv2D(32,(3,3), activation='relu', input_shape =  (48,48,1)))
model.add(tk.layers.MaxPooling2D(2,2))
model.add(tk.layers.BatchNormalization())
model.add(tk.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tk.layers.MaxPooling2D((2,2)))
model.add(tk.layers.BatchNormalization())
model.add(tk.layers.Conv2D(128,(3,3), activation='relu'))
model.add(tk.layers.Dropout(0.1))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 23, 23, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                        

In [ ]:
model.add(tk.layers.Flatten())
model.add(tk.layers.Dense(128,activation='relu'))
model.add(tk.layers.Dense(7,activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 23, 23, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                        

In [ ]:
model.compile(optimizer = tk.optimizers.Adam(learning_rate=.0001),loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
import os
try:
  os.mkdir('checkpoint')
except:
  pass
#mkdir() method in Python is used to create a directory named path with the specified numeric mode. 

In [ ]:
file_name = 'base_model.h5'
checkpoint_path= os.path.join('checkpoint',file_name)

call_back = tk.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                               monitor='val_accuracy',
                                               verbose=1,
                                               save_freq='epoch',
                                               save_best_only=True,
                                               save_weights_inly=False,
                                               mode='max')

In [ ]:
model.fit(x_train,y_train,epochs=15,validation_split=.04,callbacks=call_back)

Epoch 1/15
861/862 [============================>.] - ETA: 0s - loss: 1.5545 - accuracy: 0.3939
Epoch 00001: val_accuracy improved from -inf to 0.46214, saving model to checkpoint/base_model.h5
862/862 [==============================] - 125s 144ms/step - loss: 1.5544 - accuracy: 0.3939 - val_loss: 1.4280 - val_accuracy: 0.4621
Epoch 2/15
861/862 [============================>.] - ETA: 0s - loss: 1.3144 - accuracy: 0.5004
Epoch 00002: val_accuracy improved from 0.46214 to 0.51697, saving model to checkpoint/base_model.h5
862/862 [==============================] - 103s 119ms/step - loss: 1.3142 - accuracy: 0.5006 - val_loss: 1.2989 - val_accuracy: 0.5170
Epoch 3/15
861/862 [============================>.] - ETA: 0s - loss: 1.1799 - accuracy: 0.5521
Epoch 00003: val_accuracy improved from 0.51697 to 0.52742, saving model to checkpoint/base_model.h5
862/862 [==============================] - 99s 115ms/step - loss: 1.1798 - accuracy: 0.5521 - val_loss: 1.2870 - val_accuracy: 0.5274
Epoch 4/

In [ ]:
final_model = tk.models.load_model(checkpoint_path)
from IPython.display import clear_output
import time
for k in range(12):
  print(f'actual label is {label_to_text[y_test[k]]}')
  predicted_class = final_model.predict(tf.expand_dims(x_test[k],0)).argmax()
  print(f'predicted label is {label_to_text[predicted_class]}')
  plt.imshow(x_test[k].reshape((48,48)))
  plt.show()
  time.sleep(2)
  clear_output(wait=True)

NameError: ignored